# TP à rendre 2 Egalisation d'histogradmes
Dans ce TP, nous aimerions améliorer le contraste d'images par deux méthodes. Une méthode d'expansion de la dynamique, et une méthode d'égalisaton d'histogramme. La première image à traiter est dans un fichier npy à loader. Le reste des images sont dans le même dossier. 

In [36]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import math

# Lire et afficher l'image à utiliser

In [ ]:
image = np.genfromtxt('image.csv', delimiter=';')


In [ ]:
plt.imshow(image,cmap='gray')

# Calculer les métriques sur l'image
1.	Calculer la moyenne de luminance de l’image.
2.	Donner la dynamique de l’image (intervalle [Lmin, Lmax]).
3.	Calculer le contraste de l’image.
4.	Calculer l’écart type des niveaux de gris de l’image.


In [ ]:
def Metriques(image) :
    #moyenne luminance 
    moy_lum = 0
    for l in image :
        for p in l :
            moy_lum += p
    moy_lum /= (len(image)*len(image[0]))
    

    #dynamique de l'image
    Lmin = np.amin(image)
    Lmax = np.amax(image)
    

    #contraste
    
    contraste = (Lmax - Lmin) / (Lmax + Lmin)

    #L'écart type des niveaux de gris
    ecart_type=0
    for l in image :
        for p in l :
            ecart_type += (p - moy_lum)**2

    ecart_type /= (len(image)*len(image[0]))
    ecart_type = ecart_type**(1/2)
    return moy_lum,Lmin,Lmax,contraste,ecart_type

moy_lum,Lmin,Lmax,contraste,ecart_type=Metriques(image)
print("La moyenne de luminance = " + str(moy_lum)) 
print("La dynamique de l'image ["+str(Lmin)+","+str(Lmax)+"]")   
print("Le contraste : " + str(contraste))
print("L'ecart type : " + str(ecart_type))

# Histogramme et histogramme cumulé
5.	Construire l’histogramme de cette image ainsi que son histogramme normalisé.
6.	Construire l’histogramme cumulé de cette image ainsi que son histogramme cumulé normalisé.


In [ ]:
#une fonction qui peut generer tous les histogrammes d'une image 
def GenerateHistograms(image):
        
        image = image.astype(np.uint8)
        #Histogramme 
        hist_x=np.arange(256)
        hist_y=np.bincount(image.flatten(), minlength=256)

        #Histogramme Normlisé
        hist_n_y=[x/(len(image)*len(image[0])) for x in hist_y]

        #Histogramme cummulé
        hist_c_y=[hist_y[0]]
        for i in range(1,len(hist_y)) :
                hist_c_y.append(hist_c_y[i-1] + hist_y[i])          

        #Histogramme cummulé normalisé
        hist_cn_y=[x/(len(image)*len(image[0])) for x in hist_c_y]

        return hist_x,hist_y,hist_n_y,hist_c_y,hist_cn_y   


hist_x,hist_y,hist_n_y,hist_c_y,hist_cn_y=GenerateHistograms(image)   

plt.bar(hist_x, hist_y, color ='red',width = 0.4)
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("Histogramme")
plt.show()

plt.bar(hist_x,hist_n_y, color ='blue',width = 0.4)
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("Histogramme Normalisé")
plt.show()


plt.bar(hist_x,hist_c_y, color ='green',width = 0.4)
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("Histogramme cummulé")
plt.show()

print(hist_cn_y)
plt.bar(hist_x,hist_cn_y, color ='orange',width = 0.4)
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("Histogramme cummulé Normalisé")
plt.show()


# Comparer votre implémentation aux fonctions de opencv

In [ ]:


#l'implementation précedente
plt.bar(hist_x, hist_y, color ='red',width = 0.7)
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("l'implementation précedente ")
plt.show()

#cv2.calcHist
image=image.astype(np.uint8)
calchist_y=cv2.calcHist([image],[0],None,[256],[0,256])
calchist_y=calchist_y.astype(np.uint8)
calchist_y=calchist_y.reshape(-1)

plt.bar(np.arange(0,256),calchist_y, color ='orange',width = 0.7)
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("cv2.calcHist")
plt.show()

"""
la différence entre notre implémentation et la fonction 
cv2.calcHist est dans l'échelle sur l'axe des x,
calcHist a traité toutes les valeurs de niveaux de gris de 0 à 255
"""

#plt.hist
plt.hist(image.reshape(-1),256,[0,256])
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("plt.hist ")
plt.show()



- Expliquer la difference.
- comparer aussi avec les méthodes des autres librairies.

# 7.	Etendre l’histogramme et visualiser l’image résultante.

In [ ]:
"""Une fonction pour etendre l'histogramme et construire la nouvelle image"""
def AmeliorationContraste(image,hist_cn_y):
   
    #L'égalisation d'histogramme
    hist_eg_y=[x*255 for x in hist_cn_y]
    
    #L'image resultante
    image2=np.zeros([len(image),len(image[0])],dtype=np.uint8)
    for i in range(len(image)) :
        for j in range(len(image[0])) :
                image2[i,j]=hist_eg_y[image[i,j]]
                
    return  hist_eg_y,image2 

hist_eg_y,image2=AmeliorationContraste(image,hist_cn_y)
print(image2)
plt.bar(hist_x,hist_eg_y, color ='orange',width = 0.4)
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("Histogramme égalisé")
plt.show()


plt.imshow(image,cmap='gray')
plt.show()
plt.imshow(image2,cmap='gray')
plt.show()


# 8.	Comparer les histogrammes des deux images ; originale et étendue

In [ ]:
#Histogramme
plt.bar(hist_x, hist_y, color ='red',width = 0.5)
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("Histogramme")
plt.show()

"""
dans l'histogramme etendu en remarque que l'axe y 
représente les niveaux de gris de la nouvelle image
"""
#Histogramme égalisé
plt.bar(hist_x,hist_eg_y, color ='orange',width = 0.5)
plt.xlabel("Niveau de gris")
plt.ylabel("N° occurence")
plt.title("Histogramme égalisé")
plt.show()




# Expansion de la dynamique et comparaison avec l'égalisation d'histogramme

In [ ]:
#Expansion de la dynamique
def ExpansionDynamique(image,Lmin,Lmax):
    image3=np.zeros([len(image),len(image[0])],dtype=np.uint8)
    for i in range(len(image)) :
        for j in range(len(image[0])) :
            image3[i,j]=(255)/(Lmax-Lmin)*(image[i,j]-Lmin)
    return image3

image3=ExpansionDynamique(image,Lmin,Lmax)
print(image3)
plt.imshow(image,cmap='gray')
plt.show()
plt.imshow(image2,cmap='gray')
plt.show()
plt.imshow(image3,cmap='gray')
plt.show()


# Améliorer le contraste des images données
améliorer le contraste des 3 images envoyées avec les méthodes implémentées precedemment.

In [ ]:
# image_test1=cv2.imread('Fig0208(a).tif',0)
# image_test1=np.asarray(image_test1)
# _,Lmin,Lmax,_,_=Metriques(image_test1)
# _,_,_,_,hist_cn_y=GenerateHistograms(image_test1)
# _,image_test12=AmeliorationContraste(image_test1,hist_cn_y)
# image_test13=ExpansionDynamique(image_test1,Lmin,Lmax)

# plt.imshow(image_test1,cmap='gray')
# plt.show()
# plt.imshow(image_test12,cmap='gray')
# cv2.imwrite('Fig0208(a)_2.tif',image_test12)
# plt.show()
# plt.imshow(image_test13,cmap='gray')
# cv2.imwrite('Fig0208(a)_3.tif',image_test13)
# plt.show()

# image_test2=cv2.imread('Fig0210(a).tif',0)
# image_test2=np.asarray(image_test2)
# _,Lmin,Lmax,_,_=Metriques(image_test2)
# _,_,_,_,hist_cn_y=GenerateHistograms(image_test2)
# _,image_test22=AmeliorationContraste(image_test2,hist_cn_y)
# image_test23=ExpansionDynamique(image_test2,Lmin,Lmax)
# plt.imshow(image_test2,cmap='gray')
# plt.show()
# plt.imshow(image_test22,cmap='gray')
# cv2.imwrite('Fig0210(a)_2.tif',image_test22)
# plt.show()
# plt.imshow(image_test23,cmap='gray')
# cv2.imwrite('Fig0210(a)_3.tif',image_test23)
# plt.show()


# image_test3=cv2.imread('Fig0228(a).tif',0)
# image_test3=np.asarray(image_test3)
# _,Lmin,Lmax,_,_=Metriques(image_test3)
# _,_,_,_,hist_cn_y=GenerateHistograms(image_test3)
# _,image_test32=AmeliorationContraste(image_test3,hist_cn_y)
# image_test33=ExpansionDynamique(image_test3,Lmin,Lmax)
# plt.imshow(image_test3,cmap='gray')
# plt.show()
# plt.imshow(image_test32,cmap='gray')
# cv2.imwrite('Fig0228(a)_2.tif',image_test32)
# plt.show()
# plt.imshow(image_test33,cmap='gray')
# cv2.imwrite('Fig0228(a)_3.tif',image_test33)
# plt.show()


# Gradiant

In [ ]:
image2 = np.genfromtxt('mat.csv', delimiter=';')
print(image2)
h,w = image2.shape
imageG = np.zeros([h,w])
imageG=cv2.copyMakeBorder(image2,1,1,1,1,cv2.BORDER_CONSTANT,(0,0,0))

#gradient x
gx = np.zeros([h,w])
for p in range(1,w+1) :
    for l in range(1,h+1) :
        gx[l-1,p-1] = imageG[l+1,p] - imageG[l,p]
print(gx)
#gradient y
gy = np.zeros([h,w])
for l in range(1,h+1) :
    for p in range(1,w+1) :
        gy[l-1,p-1] = imageG[l,p+1] - imageG[l,p]
print(gy)

#Mod
m = np.zeros([h,w])
for l in range(h) :
    for p in range(w) :
        m[l,p]= math.sqrt( gx[l,p]**2 + gy[l,p]**2 );
print(m)
#direction
d = np.zeros([h,w])
for l in range(h) :
    for p in range(w) :
        d[l,p]= math.degrees(math.atan( gy[l,p] / gx[l,p]));
print(d)




# Binarisation

In [ ]:
seuil = 70
imageBin = np.zeros([h,w])
for l in range(h) :
    for p in range(w) :
        imageBin[l,p] = int( m[l,p] > seuil)
print(m)
print(imageBin)    




# Dilatation

In [ ]:
#dilatation
dil=np.copy(imageBin)
for i in range(1,h-1):
    for j in range(1,w-1):
        k=imageBin[i,j] + imageBin[i-1,j] + imageBin[i+1,j] + imageBin[i,j-1]+ imageBin[i,j+1]
        if(k>0):
            k=1
        dil[i,j]=k    
        dil[i-1,j]=k
        dil[i+1,j]=k 
        dil[i,j-1]=k
        dil[i,j+1]=k
print(imageBin)
print(dil)
plt.imshow(dil,cmap='gray')
    

# erosion

In [ ]:
#erosione
er=np.copy(imageBin)
for i in range(0,h-2):
    for j in range(0,w-2):
        k=1
        for a in range(i,i+3):
            for b in range(j,j+3):
                 k=k*imageBin[a,b]
        for a in range(i,i+3):
            for b in range(j,j+3):
                er[a,b]=k
        
print(imageBin)
print(er)               
        

# Prewitt & Sobel

In [ ]:
#prewitt
c=1
imagepr = np.zeros([h,w])
imagepr=cv2.copyMakeBorder(image2,1,1,1,1,cv2.BORDER_CONSTANT,(0,0,0))
cx=[[-1,0,1],[-c,0,c],[-1,0,1]]
cy=[[-1,-c,-1],[0,0,0],[1,c,1]]

prewitt_x=np.zeros([h,w])
prewitt_y=np.zeros([h,w])
for i in range (1,h+1) :
    for j in range(1,w+1):
        for ki in range(3):
            for kj in range(3):
                prewitt_x[i-1,j-1]+=(imagepr[i-1+ki,j-1+kj]*cx[ki][kj])
                prewitt_y[i-1,j-1]+=(imagepr[i-1+ki,j-1+kj]*cy[ki][kj])

print(prewitt_x)
print(prewitt_y)

#sobel
sobel_x=np.zeros([h,w])
sobel_y=np.zeros([h,w])
c=2
cx=[[-1,0,1],[-c,0,c],[-1,0,1]]
cy=[[-1,-c,-1],[0,0,0],[1,c,1]]
for i in range (1,h+1) :
    for j in range(1,w+1):
        for ki in range(3):
            for kj in range(3):
                sobel_x[i-1,j-1]+=imagepr[i-1+ki,j-1+kj]*cx[ki][kj]
                sobel_y[i-1,j-1]+=imagepr[i-1+ki,j-1+kj]*cy[ki][kj]

print(sobel_x)
print(sobel_y)


# laplacian

In [ ]:
#laplacian
lp_mask=[[0,1,0],[1,-4,1],[0,1,0]]
lp=np.zeros([h,w])
print(imagepr)
for i in range (1,h+1) :
    for j in range(1,w+1):
        for ki in range(3):
            for kj in range(3):
                lp[i-1,j-1]+=imagepr[i-1+ki,j-1+kj]*lp_mask[ki][kj]
                
print(lp)



# Filtre Moyen

In [33]:
#filtre moyen
fm=np.zeros([h,w])
for i in range (1,h+1) :
    for j in range(1,w+1):
        for ki in range(3):
            for kj in range(3):
                fm[i-1,j-1]+=imagepr[i-1+ki,j-1+kj]
        fm[i-1][j-1]/= 9                       
print(fm)


[[ 11.22222222  18.44444444  40.88888889  81.44444444 102.55555556
   96.66666667  74.22222222  45.88888889]
 [ 27.88888889  45.44444444  87.22222222 146.22222222 182.22222222
  157.         124.22222222  70.66666667]
 [ 50.88888889  78.77777778 136.33333333 187.66666667 233.55555556
  182.22222222 150.77777778  77.        ]
 [ 70.22222222 109.77777778 163.33333333 202.44444444 237.55555556
  195.22222222 157.          82.33333333]
 [ 87.66666667 120.88888889 158.33333333 177.66666667 204.22222222
  167.22222222 138.44444444  78.66666667]
 [100.22222222 125.33333333 131.22222222 117.33333333 131.77777778
  112.33333333 117.66666667  78.11111111]
 [110.11111111 136.66666667 124.44444444  88.55555556  96.66666667
   97.11111111 130.88888889  96.22222222]
 [ 76.          98.55555556  83.11111111  48.55555556  50.33333333
   64.77777778  99.44444444  75.11111111]]


# Filtre gaussien

In [37]:
#filtre gaussien 
_,_,_,_,ecrt=Metriques(image2) 
fg=np.zeros([h,w])
for x in range(h):
    for y in range(w):
        fg[x,y]=1/(2*math.pi*2*(ecrt**2)) * math.exp((-(x**2)-(y**2) ) /(2*(ecrt**2)))
print(fg)        

SyntaxError: '(' was never closed (Temp/ipykernel_10864/3311187653.py, line 5)

# Consignes
Le travail est à remettre par groupe de 4 au maximum [1..4].

Le délai est le vendredi à 19h


In [ ]:
# bonne chance